# Relevant Segment Extraction (RSE) for Enhanced RAG

## Setting Up the Environment

In [35]:
import fitz
import numpy as np
import json
import re

In [36]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [37]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
    pdf_path (str): PDF 파일의 경로입니다.

    Returns:
    str: PDF에서 추출한 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = "" # 추출된 텍스트를 저장할 빈 문자열을 초기화합니다.

    # PDF의 각 페이지를 반복합니다.
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num] # 페이지 가져오기
        text = page.get_text("text") # 페이지에서 텍스트 추출
        all_text += text # 추출한 텍스트를 all_text 문자열에 추가합니다.

    return all_text # 추출된 텍스트를 반환합니다.

## Chunking the Extracted Text

In [38]:
def chunk_text(text, chunk_size=800, overlap=0):
    """
    텍스트를 겹침(overlap) 없이 일정한 크기로 분할합니다.
    RSE(Retrieval Segment Evaluation)에서는 겹치지 않는 청크가 일반적으로 필요합니다.
    
    Args:
        text (str): 분할할 원본 텍스트
        chunk_size (int): 각 청크의 문자 길이 (기본값: 800자)
        overlap (int): 청크 간 겹치는 문자 수 (기본값: 0)
        
    Returns:
        List[str]: 분할된 텍스트 청크 리스트
    """
    chunks = []

    # 문자 기준으로 일정 간격마다 슬라이싱하며 청크 생성
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]
        if chunk:  # 빈 청크가 아닌 경우에만 추가
            chunks.append(chunk)

    return chunks

## Building a Simple Vector Store

In [39]:
class SimpleVectorStore:
    """
    NumPy를 활용한 간단한 벡터 저장소 구현체입니다.
    """
    def __init__(self, dimension=1536):
        """
        벡터 저장소 초기화
        
        Args:
            dimension (int): 임베딩 벡터의 차원 수
        """
        self.dimension = dimension
        self.vectors = []     # 임베딩 벡터 리스트
        self.documents = []   # 문서(청크) 리스트
        self.metadata = []    # 각 문서의 메타데이터 리스트

    def add_documents(self, documents, vectors=None, metadata=None):
        """
        문서와 벡터를 벡터 저장소에 추가
        
        Args:
            documents (List[str]): 문서 청크 리스트
            vectors (List[List[float]], optional): 각 문서의 임베딩 벡터 리스트
            metadata (List[Dict], optional): 각 문서에 대한 메타데이터 리스트
        """
        if vectors is None:
            vectors = [None] * len(documents)

        if metadata is None:
            metadata = [{} for _ in range(len(documents))]

        for doc, vec, meta in zip(documents, vectors, metadata):
            self.documents.append(doc)
            self.vectors.append(vec)
            self.metadata.append(meta)

    def search(self, query_vector, top_k=5):
        """
        가장 유사한 문서를 검색 (코사인 유사도 기준)
        
        Args:
            query_vector (List[float]): 질의에 대한 임베딩 벡터
            top_k (int): 반환할 상위 결과 개수
            
        Returns:
            List[Dict]: 문서, 유사도 점수, 메타데이터가 포함된 결과 리스트
        """
        if not self.vectors or not self.documents:
            return []

        # 질의 벡터를 NumPy 배열로 변환
        query_array = np.array(query_vector)

        # 각 벡터와의 코사인 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            if vector is not None:
                similarity = np.dot(query_array, vector) / (
                    np.linalg.norm(query_array) * np.linalg.norm(vector)
                )
                similarities.append((i, similarity))

        # 유사도 기준으로 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 상위 top_k 결과 추출
        results = []
        for i, score in similarities[:top_k]:
            results.append({
                "document": self.documents[i],
                "score": float(score),
                "metadata": self.metadata[i]
            })

        return results

## Creating Embeddings for Text Chunks

In [40]:
def create_embeddings(texts, model="text-embedding-3-small"):
    """
    텍스트 리스트에 대해 임베딩 벡터를 생성합니다.
    
    Args:
        texts (List[str]): 임베딩할 텍스트들의 리스트
        model (str): 사용할 임베딩 모델 이름
        
    Returns:
        List[List[float]]: 임베딩 벡터 리스트
    """
    if not texts:
        return []  # 텍스트가 없으면 빈 리스트 반환

    # 긴 리스트일 경우 배치 단위로 처리
    batch_size = 100  # API 제한에 따라 조정 가능
    all_embeddings = []  # 전체 임베딩을 저장할 리스트 초기화

    # 텍스트 리스트를 배치 단위로 나눠 임베딩 처리
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # 현재 배치 추출

        # 지정된 모델을 사용하여 배치 임베딩 생성
        response = client.embeddings.create(
            input=batch,
            model=model
        )

        # 응답에서 임베딩 벡터 추출
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 전체 리스트에 추가

    return all_embeddings  # 모든 임베딩 결과 반환

## Processing Documents with RSE

In [41]:
def process_document(pdf_path, chunk_size=800):
    """
    RSE(Retrieval Segment Evaluation) 또는 RAG에 사용할 수 있도록 문서를 처리합니다.

    Args:
        pdf_path (str): PDF 문서의 경로.
        chunk_size (int): 각 청크의 문자 길이.

    Returns:
        Tuple[List[str], SimpleVectorStore, Dict]:
            - 분할된 텍스트 청크 리스트.
            - 벡터 저장소 객체(SimpleVectorStore).
            - 문서 정보 (청크와 소스 경로 포함).
    """
    print("문서에서 텍스트를 추출 중...")
    text = extract_text_from_pdf(pdf_path)

    print("텍스트를 중첩 없이 청크 단위로 분할 중...")
    chunks = chunk_text(text, chunk_size=chunk_size, overlap=0)
    print(f"{len(chunks)}개의 청크가 생성되었습니다.")

    print("청크 임베딩을 생성 중...")
    chunk_embeddings = create_embeddings(chunks)

    # 벡터 저장소 생성
    vector_store = SimpleVectorStore()

    # 각 청크에 대한 메타데이터 생성
    metadata = [{"chunk_index": i, "source": pdf_path} for i in range(len(chunks))]

    # 청크와 임베딩을 저장소에 추가
    vector_store.add_documents(chunks, chunk_embeddings, metadata)

    # 문서 정보 저장 (재구성에 사용할 수 있음)
    doc_info = {
        "chunks": chunks,
        "source": pdf_path,
    }

    return chunks, vector_store, doc_info

## RSE Core Algorithm: Computing Chunk Values and Finding Best Segments

In [42]:
def calculate_chunk_values(query, chunks, vector_store, irrelevant_chunk_penalty=0.2):
    """
    질의에 대한 관련성과 위치 정보를 바탕으로 각 청크의 가치를 계산합니다.
    
    Args:
        query (str): 사용자 질의
        chunks (List[str]): 문서 청크 리스트
        vector_store (SimpleVectorStore): 벡터 저장소 (청크 포함)
        irrelevant_chunk_penalty (float): 관련 없는 청크에 부여할 감점 값
        
    Returns:
        List[float]: 각 청크에 대한 가치 점수 리스트
    """
    # 질의 임베딩 생성
    query_embedding = create_embeddings([query])[0]

    # 벡터 저장소에서 모든 청크에 대한 유사도 검색 (최대 청크 수만큼)
    num_chunks = len(chunks)
    results = vector_store.search(query_embedding, top_k=num_chunks)

    # 검색 결과로부터 청크 인덱스별 관련성 점수 매핑 생성
    relevance_scores = {result["metadata"]["chunk_index"]: result["score"] for result in results}

    # 각 청크에 대해 가치 점수 계산 (관련성 - 감점)
    chunk_values = []
    for i in range(num_chunks):
        # 해당 청크의 관련성 점수가 없으면 기본값 0 사용
        score = relevance_scores.get(i, 0.0)
        # 감점 적용: 관련 없는 청크는 음수 값이 될 수 있음
        value = score - irrelevant_chunk_penalty
        chunk_values.append(value)

    return chunk_values

In [43]:
def find_best_segments(chunk_values, max_segment_length=20, total_max_length=30, min_segment_value=0.2):
    """
    최대 합 부분 배열 알고리즘(변형)을 사용하여 가장 가치 있는 연속 청크 구간(세그먼트)을 탐색합니다.

    Args:
        chunk_values (List[float]): 각 청크에 대한 점수 또는 가치 리스트.
        max_segment_length (int): 하나의 세그먼트가 가질 수 있는 최대 길이.
        total_max_length (int): 전체 포함 가능한 청크 수의 최대 합.
        min_segment_value (float): 세그먼트로 인정되기 위한 최소 점수 합계.

    Returns:
        Tuple[List[Tuple[int, int]], List[float]]: 
            - 세그먼트 리스트 (각각 시작 인덱스, 종료 인덱스 형태).
            - 각 세그먼트의 총 점수 리스트.
    """
    print("최적의 연속 세그먼트를 찾는 중...")

    best_segments = []         # 선택된 세그먼트 저장
    segment_scores = []        # 각 세그먼트의 총 점수
    total_included_chunks = 0  # 전체 포함된 청크 수 누적

    # 전체 길이 제한에 도달할 때까지 반복 탐색
    while total_included_chunks < total_max_length:
        best_score = min_segment_value
        best_segment = None

        for start in range(len(chunk_values)):
            # 이미 포함된 세그먼트와 겹치는 경우 제외
            if any(start >= s[0] and start < s[1] for s in best_segments):
                continue

            # 가능한 세그먼트 길이 내에서 탐색
            for length in range(1, min(max_segment_length, len(chunk_values) - start) + 1):
                end = start + length

                # 종료 지점이 기존 세그먼트와 겹치면 제외
                if any(end > s[0] and end <= s[1] for s in best_segments):
                    continue

                segment_value = sum(chunk_values[start:end])

                if segment_value > best_score:
                    best_score = segment_value
                    best_segment = (start, end)

        # 가장 가치 있는 세그먼트를 추가
        if best_segment:
            best_segments.append(best_segment)
            segment_scores.append(best_score)
            total_included_chunks += best_segment[1] - best_segment[0]
            print(f"세그먼트 {best_segment} 발견 (점수: {best_score:.4f})")
        else:
            break  # 더 이상 유효한 세그먼트 없음

    # 시작 인덱스 기준 정렬
    best_segments = sorted(best_segments, key=lambda x: x[0])

    return best_segments, segment_scores

## Reconstructing and Using Segments for RAG

In [44]:
def reconstruct_segments(chunks, best_segments):
    """
    선택된 청크 인덱스를 바탕으로 텍스트 세그먼트를 재조립합니다.
    
    Args:
        chunks (List[str]): 전체 문서를 분할한 청크 리스트
        best_segments (List[Tuple[int, int]]): (시작, 끝) 인덱스로 구성된 세그먼트 리스트
        
    Returns:
        List[str]: 재조립된 텍스트 세그먼트 리스트 (딕셔너리 형태 포함)
    """
    reconstructed_segments = []  # 재조립된 세그먼트를 저장할 리스트

    for start, end in best_segments:
        # 해당 범위의 청크들을 연결하여 하나의 세그먼트 텍스트로 생성
        segment_text = " ".join(chunks[start:end])
        
        # 세그먼트 텍스트와 인덱스 범위를 함께 저장
        reconstructed_segments.append({
            "text": segment_text,
            "segment_range": (start, end),
        })

    # 재조립된 세그먼트 리스트 반환
    return reconstructed_segments

In [45]:
def format_segments_for_context(segments):
    """
    LLM 입력용 문맥(context) 형식으로 세그먼트를 구성합니다.
    
    Args:
        segments (List[Dict]): 세그먼트 딕셔너리들의 리스트
        
    Returns:
        str: 포맷팅된 문맥 문자열
    """
    context = []  # 포맷팅된 문맥 문자열 조각들을 담을 리스트 초기화

    for i, segment in enumerate(segments):
        # 각 세그먼트에 대한 헤더 생성 (세그먼트 번호 및 청크 범위 표시)
        segment_header = f"SEGMENT {i+1} (Chunks {segment['segment_range'][0]}-{segment['segment_range'][1]-1}):"
        context.append(segment_header)          # 헤더 추가
        context.append(segment['text'])         # 세그먼트 텍스트 추가
        context.append("-" * 80)                # 가독성을 위한 구분선 추가

    # 리스트의 모든 요소를 두 줄 간격으로 이어붙여 최종 문맥 문자열 생성
    return "\n\n".join(context)

## Generating Responses with RSE Context

In [46]:
def generate_response(query, context, model="gpt-4o-mini"):
    """
    주어진 질의와 문맥을 바탕으로 LLM 응답을 생성합니다.

    Args:
        query (str): 사용자 질의.
        context (str): 관련 세그먼트로 구성된 문맥 텍스트.
        model (str): 사용할 LLM 모델 이름.

    Returns:
        str: 생성된 응답 텍스트.
    """
    print("관련 세그먼트를 문맥으로 활용하여 응답을 생성합니다...")

    # 시스템 프롬프트: LLM의 역할 정의
    system_prompt = """당신은 제공된 문맥을 기반으로 질문에 답하는 유용한 AI 어시스턴트입니다.
    문맥은 쿼리와 관련된 문서 세그먼트로 구성되어 있으며,
    당신은 그 정보를 활용해 정확하고 포괄적인 답변을 생성해야 합니다.
    만약 문맥에 해당 질문에 대한 직접적인 정보가 없다면, 그 사실을 명확히 언급하세요."""

    # 사용자 프롬프트: 문맥 + 질의
    user_prompt = f"""
    Context:
    {context}

    Question: {query}

    위 문맥에 따라 가능한 한 구체적이고 유익한 답변을 작성해 주세요.
    """

    # LLM 호출하여 응답 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    return response.choices[0].message.content

## Complete RSE Pipeline Function

In [47]:
def rag_with_rse(pdf_path, query, chunk_size=800, irrelevant_chunk_penalty=0.2):
    """
    Relevant Segment Extraction(RSE)을 포함한 RAG 파이프라인 전체 실행 함수입니다.

    Args:
        pdf_path (str): 처리할 PDF 문서의 경로.
        query (str): 사용자 질의.
        chunk_size (int): 분할할 청크의 문자 수.
        irrelevant_chunk_penalty (float): 관련 없는 청크에 부과할 감점 값.

    Returns:
        Dict: 다음 정보를 포함한 결과 딕셔너리:
            - query: 사용자 질의.
            - segments: 선택된 세그먼트 텍스트.
            - response: LLM이 생성한 응답.
    """
    print("\n***RAG with Relevant Segment Extraction 시작***")
    print(f"사용자 질문: {query}")

    # 1. 문서 전처리 (PDF → 텍스트 추출 → 청크 분할 → 임베딩 생성)
    chunks, vector_store, doc_info = process_document(pdf_path, chunk_size)

    # 2. 각 청크에 대해 쿼리 기반 관련성 점수 계산
    print("\n관련성 점수 계산 중...")
    chunk_values = calculate_chunk_values(
        query=query,
        chunks=chunks,
        vector_store=vector_store,
        irrelevant_chunk_penalty=irrelevant_chunk_penalty
    )

    # 3. 가장 높은 점수를 가진 연속 세그먼트 선택
    best_segments, scores = find_best_segments(
        chunk_values,
        max_segment_length=20,
        total_max_length=30,
        min_segment_value=0.2
    )

    # 4. 선택된 세그먼트를 기반으로 문맥 구성
    print("\n세그먼트 재구성 중...")
    segments = reconstruct_segments(chunks, best_segments)

    # 5. LLM 입력용 문맥 문자열 포맷팅
    context = format_segments_for_context(segments)

    # 6. 문맥 + 질의를 기반으로 응답 생성
    response = generate_response(query, context)

    # 7. 결과 정리
    result = {
        "query": query,
        "segments": segments,
        "response": response
    }

    print("\n***최종 응답 결과***")
    print(response)

    return result

## Comparing with Standard Retrieval

In [48]:
def standard_top_k_retrieval(pdf_path, query, k=10, chunk_size=800):
    """
    상위 k개의 청크를 검색하여 문맥으로 사용하는 표준 RAG 방식입니다.
    
    Args:
        pdf_path (str): PDF 문서 경로
        query (str): 사용자 질의
        k (int): 검색할 상위 관련 청크 개수
        chunk_size (int): 청크 크기 (문자 단위)
        
    Returns:
        Dict: 질의, 검색된 청크, 생성된 응답이 포함된 결과
    """
    print("\n=== STARTING STANDARD TOP-K RETRIEVAL ===")
    print(f"Query: {query}")

    # 문서 전처리: 텍스트 추출, 청크 분할, 임베딩 생성
    chunks, vector_store, doc_info = process_document(pdf_path, chunk_size)

    # 질의 임베딩 생성 후, 상위 k개 청크 검색
    print("Creating query embedding and retrieving chunks...")
    query_embedding = create_embeddings([query])[0]
    results = vector_store.search(query_embedding, top_k=k)

    # 검색된 청크 텍스트만 추출
    retrieved_chunks = [result["document"] for result in results]

    # 검색된 청크들을 문맥 문자열로 포맷팅
    context = "\n\n".join([
        f"CHUNK {i+1}:\n{chunk}" 
        for i, chunk in enumerate(retrieved_chunks)
    ])

    # 문맥을 기반으로 LLM 응답 생성
    response = generate_response(query, context)

    # 결과 정리
    result = {
        "query": query,
        "chunks": retrieved_chunks,
        "response": response
    }

    print("\n=== FINAL RESPONSE ===")
    print(response)

    return result

## Evaluation of RSE

In [49]:
def evaluate_methods(pdf_path, query, reference_answer=None):
    """
    RSE 방식과 표준 Top-K 검색 방식을 비교 평가합니다.

    Args:
        pdf_path (str): PDF 문서 경로.
        query (str): 사용자 질문.
        reference_answer (str, optional): 기준 정답 (있을 경우 응답 평가 수행).

    Returns:
        Dict: RSE 방식과 표준 방식의 결과를 포함한 딕셔너리.
    """
    print("\n========= 평가 시작 =========\n")

    # 1. Relevant Segment Extraction 기반 RAG 실행
    rse_result = rag_with_rse(pdf_path, query)

    # 2. 표준 Top-K 검색 기반 RAG 실행
    standard_result = standard_top_k_retrieval(pdf_path, query)

    # 3. 기준 정답이 주어졌을 경우, 두 응답을 비교 평가
    if reference_answer:
        print("\n=== 응답 비교 평가 ===")

        evaluation_prompt = f"""
    Query: {query}

    [Reference Answer]
    {reference_answer}

    [Standard Retrieval Response]
    {standard_result['response']}

    [Relevant Segment Extraction Response]
    {rse_result['response']}

    위 두 응답을 기준 정답과 비교하여 다음을 판단하세요:
    1. 더 정확하고 포괄적인 응답은 무엇인가요?
    2. 사용자 질문을 더 잘 해결한 응답은 어떤 것인가요?
    3. 불필요하거나 관련 없는 정보를 덜 포함한 응답은 무엇인가요?

    각 항목에 대해 근거를 명확히 설명한 후, 어느 방식이 더 우수했는지 종합적으로 평가하세요.
        """

        print("기준 정답과 비교하여 응답 평가 중...")

        evaluation = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 RAG 응답을 공정하게 평가하는 시스템입니다."},
                {"role": "user", "content": evaluation_prompt}
            ]
        )

        print("\n=== 평가 결과 ===")
        print(evaluation.choices[0].message.content)

    return {
        "rse_result": rse_result,
        "standard_result": standard_result
    }


In [50]:
# 검증용 JSON 파일에서 데이터 로드
with open('dataset/validation.json') as f:
    data = json.load(f)

# 첫 번째 테스트 케이스의 질의 추출
query = data[0]['question']

# 참조 정답(ideal_answer) 추출
reference_answer = data[0]['ideal_answer']

# 사용할 PDF 문서 경로
pdf_path = "dataset/AI_Understanding.pdf"

# 두 가지 RAG 방식(RSE vs Standard Top-K)에 대한 평가 실행
results = evaluate_methods(pdf_path, query, reference_answer)


========= 평가 시작 =========


***RAG with Relevant Segment Extraction 시작***
사용자 질문: '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?
문서에서 텍스트를 추출 중...
텍스트를 중첩 없이 청크 단위로 분할 중...
21개의 청크가 생성되었습니다.
청크 임베딩을 생성 중...

관련성 점수 계산 중...
최적의 연속 세그먼트를 찾는 중...
세그먼트 (0, 20) 발견 (점수: 5.5964)
세그먼트 (20, 21) 발견 (점수: 0.2936)

세그먼트 재구성 중...
관련 세그먼트를 문맥으로 활용하여 응답을 생성합니다...

***최종 응답 결과***
설명 가능한 AI(Explainable AI, XAI)는 AI 시스템이 의사 결정을 내리는 방식을 투명하고 이해하기 쉽게 만드는 것을 목표로 하는 기술입니다. XAI는 AI 모델의 작동 원리를 설명하고, 사용자가 AI의 결정을 평가할 수 있도록 돕는 데 중점을 둡니다. 이는 AI 시스템이 '블랙박스'처럼 작동하여 그 결정 과정이 불투명한 문제를 해결하기 위한 접근 방식입니다.

설명 가능한 AI가 중요한 이유는 다음과 같습니다:

1. **신뢰 구축**: AI 시스템의 결정 과정이 명확하게 설명되면 사용자와 이해관계자들이 AI에 대한 신뢰를 가질 수 있습니다. 신뢰는 AI의 광범위한 채택과 긍정적인 사회적 영향을 위해 필수적입니다.

2. **책임성**: AI의 결정이 어떻게 이루어졌는지를 이해함으로써, 개발자와 사용자 모두가 AI 시스템의 결과에 대한 책임을 질 수 있습니다. 이는 윤리적이고 책임감 있는 AI 사용을 촉진합니다.

3. **편견 및 공정성**: AI 시스템이 내리는 결정이 공정한지 평가하기 위해서는 그 결정의 근거를 이해해야 합니다. XAI는 AI의 편향성을 식별하고 수정하는 데 도움을 줄 수 있습니다.

4. **규제 준수**: 많은 산업에서 AI의 사용이 증가함에 따라, 규제 기관은 AI 시스템의 투명